## Урок 8. GAN
###  - Обучите нейронную сеть любой архитектуры, которой не было на курсе, либо нейронную сеть разобранной архитектуры, <br>но на том датасете, которого не было на уроках. <br> - Сделайте анализ того, что вам помогло<br> в улучшения работы нейронной сети

In [ ]:
# !pip install opendatasets

In [1]:
import opendatasets as od

In [2]:
od.download("https://github.com/JurgenPalsma/swarmex/archive/refs/heads/master.zip")

20684800it [00:13, 1512069.36it/s]


In [2]:
!unzip "/neyroseti/swarmex-master.zip" -d "/neyroseti/"

unzip:  cannot find or open /neyroseti/swarmex-master.zip, /neyroseti/swarmex-master.zip.zip or /neyroseti/swarmex-master.zip.ZIP.


In [3]:
%cd swarmex-master/

/home/semen/Ucheba/DZ/II/neyroseti/swarmex-master


In [ ]:
# !pip install py4j

In [4]:
!python main.py -c ./config/demo.json

/bin/bash: строка 1: python: команда не найдена


Решил использовать нейросеть с использованием _partical swarm optimization_.
Метод роя частиц (МРЧ) — метод численной оптимизации, для использования которого не требуется знать точного градиента оптимизируемой функции.
МРЧ был доказан Кеннеди, Эберхартом и Ши[1][2] и изначально предназначался для имитации социального поведения. Алгоритм был упрощён, и было замечено, что он пригоден для выполнения оптимизации. Книга Кеннеди и Эберхарта[3] описывает многие философские аспекты МРЧ и так называемого роевого интеллекта. Обширное исследование приложений МРЧ сделано Поли[4][5]. МРЧ оптимизирует функцию, поддерживая популяцию возможных решений, называемых частицами, и перемещая эти частицы в пространстве решений согласно простой формуле. Перемещения подчиняются принципу наилучшего найденного в этом пространстве положения, которое постоянно изменяется при нахождении частицами более выгодных положений.  
Для применения модели выбрал работу JurgenPalsma на [github](https://github.com/JurgenPalsma/swarmex).  
Полноценный тестовый запус осуществить не удалось, поэтому обратился к другому источнику.

In [ ]:
# !pip install pyswarms

In [5]:
%cd ..
!pwd

/home/semen/Ucheba/DZ/II/neyroseti
/home/semen/Ucheba/DZ/II/neyroseti


In [1]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.datasets import load_iris


# Import PySwarms
import pyswarms as ps


%load_ext autoreload
%autoreload 2

In [2]:
# список признаков, по которым будем учить
channelIndexes = [0, 1, 2, 3]
#channelIndexes = [0,1,2]

# длина истории для работы
xLen = 8
# отступ тестов от тренировок
bias = 4

# шаг по данным для построения обучающих примеров
step = 1

# горизонт предсказания
future = 1


# параметры модели для обучения
batch_size = 8
epochs = 5

train_split_percents = 80

In [3]:
def data_get(path_to_csv, start_date, batch_size):
  df = pd.read_csv(path_to_csv, usecols=['<DATE>','<HIGH>','<LOW>'],
                   index_col=['<DATE>'], parse_dates=['<DATE>'],) 
  df = df[df.index >= pd.to_datetime(start_date)]
  df["HLAvg"] = df['<HIGH>'].add(df['<LOW>']).div(2)
  del df['<HIGH>']
  del df['<LOW>']

  # Простое скользящее среднее
  df['MA'] = df['HLAvg'].rolling(window=14).mean()
  # Логарифмический возврат
  df['Returns'] = np.log(df['MA']/df['MA'].shift(1))
  
  df.dropna(how='any', inplace=True)

  return df

#### Попытка применения нейросети на данных с котировками CHFUSD

In [5]:
df = data_get('./CHFUSD_210101_220118_1H.csv', '01.01.2021', batch_size)
# Простое скользящее среднее"
df['MA'] = df['HLAvg'].rolling(window=xLen).mean()
# Log Returns
df['Returns'] = np.log(df['MA']/df['MA'].shift(1))
df.dropna(how='any', inplace=True)
df = df[df.shape[0] % batch_size:]
validation_size = round(df.shape[0]*0.2)
test_size = round(df.shape[0]*0.2)
df_train = df[:- validation_size - test_size]
df_validation = df[- validation_size - test_size - xLen:- test_size]
df_test = df[- test_size - xLen:]
df_train.shape, df_validation.shape, df_test.shape

((4066, 3), (1369, 3), (1369, 3))

In [6]:
def get_train(values, window_size):
    X, y = [], []
    len_values = len(values)
    for i in range(window_size, len_values):
        X.append(values[i-window_size:i])
        y.append(values[i])
    X, y = np.asarray(X), np.asarray(y)
    X = np.reshape(X, (X.shape[0], X.shape[1]))
    y = np.reshape(y, y.shape[0])
    return X, y

def get_val(values, window_size):
    X = []
    len_values = len(values)
    for i in range(window_size, len_values):
        X.append(values[i-window_size:i])
    X = np.asarray(X)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    y = values[-X.shape[0]:]
    return X, y

In [7]:
X, y = get_train(df[['Returns']].values, xLen)
y[y<-0.000001] = -1
y[y>0.000001] = 1
y = np.where((y>-0.000001)&(y<0.000001), 0, y)
y = y.astype(int)
X.shape, y.shape

((6762, 14), (6762,))

In [8]:
# Прямое распространение
def forward_prop(params):
    """Функция прямого распространения в качестве функции цели 

    Эта функция вычисляет прямое распространения в нейронной сети, а также расчитывает потерю.
    Она получает набор параметров, которые должны быть
    развернуты в соответствующие веса и смещения.

    Входные данные
    ------
    Параметры: np.ndarray
        Размерности должны включать развернутую версию
        весов и смещений.

    Возвращает
    -------
    float
        Вычисленная потеря отрицательной логарифмической вероятности при заданных параметрах.
    """
    # Архитектура нейронной сети
    n_inputs = 14
    n_hidden = 370
    n_classes = 3

    # Откат весов и смещений
    W1 = params[0:5180].reshape((n_inputs,n_hidden))
    b1 = params[5180:5550].reshape((n_hidden,))
    W2 = params[5550:6660].reshape((n_hidden,n_classes))
    b2 = params[6660:6663].reshape((n_classes,))

    # Выполнение прямого распространения
    z1 = X.dot(W1) + b1  # Предварительная активация на 1 слое
    a1 = np.tanh(z1)     # Активация на 1 слое
    z2 = a1.dot(W2) + b2 # предварительная активация на 2 слое
    logits = z2          # Логиты для 2 слоев 

    # Вычисление логитов softmax
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Вычисление отрицательной логарифмической вероятности
    N = 6762 # Количество выборок
    corect_logprobs = -np.log(probs[range(N)])
    loss = np.sum(corect_logprobs) / N

    return loss

##### Для выполнения прямого распространения в рою можно использовать более высокоуровневый метод, называемый "глобальным прямым распространением". В этом методе каждая частица в рое вычисляет свою собственную потерю индивидуально, а затем потери от всех частиц собираются для получения общей потери для всего роя.

##### Вот шаги, включенные в метод глобального прямого распространения:

##### 1. Каждая частица в рое выполняет прямое распространение независимо, используя свой собственный набор весов и смещений. Это включает передачу входных данных через модель нейронной сети и вычисление предсказанных выходных значений.

##### 2. После прямого распространения каждая частица вычисляет свою собственную потерю, сравнивая свои предсказанные выходные значения с фактическими метками с помощью функции потерь, такой как среднеквадратичная ошибка или перекрестная энтропия.

##### 3. Затем потери от всех частиц собираются и агрегируются для получения общей потери для всего роя. Это можно сделать, взяв среднее значение индивидуальных потерь или с использованием взвешенного среднего на основе приспособленности или производительности частицы.

##### 4. Общая потеря затем используется для обновления глобального лучшего положения и лучшей приспособленности роя, если необходимо, на основе функции приспособленности, которая оценивает производительность каждой частицы.

##### 5. Наконец, обновленное глобальное лучшее положение используется для обновления весов и смещений каждой частицы в рое, что позволяет им учиться на основе совокупных знаний всего роя.

##### Использование глобального прямого распространения позволяет рою эффективно исследовать пространство поиска и находить лучшие решения коллективно. Это позволяет эффективно обмениваться информацией и учиться на опыте друг друга, что приводит к улучшению производительности и скорости сходимости.

In [9]:
def f(x):
    
    
    """Более высокоуровневый метод для выполнения прямого распространения во всем рое.

    Входные данные
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
          Рой, который будет выполнять поиск

    Возвращает 
    -------
    numpy.ndarray of shape (n_particles, )
        Вычисленные потери для каждой частицы
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [10]:
# Инициализировать рой
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Вызов экземпляра PSO
dimensions = y.shape[0]
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Выполняем оптимизацию
cost, pos = optimizer.optimize(f, iters=100, verbose=3)

2023-07-17 19:35:12,562 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


pyswarms.single.global_best: 100%|██████████|100/100, best_cost=3.3
2023-07-17 20:08:11,034 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 3.295893309325902, best pos: [1.057356   1.43019339 0.82459699 ... 0.38761439 0.28801263 0.71547585]


In [11]:
def predict(X, pos):
    """
    Используем обученные веса для выполнения предсказаний классов.

    Входные данные
    ------
    X: numpy.ndarray
        Входной набор данных Iris
    pos: numpy.ndarray
        Матрица позиций, найденная роем. Будет преобразована
        в веса и смещения.
    """
    # Архитектура нейронной сети
    n_inputs = 14
    n_hidden = 370
    n_classes = 3

    # Преобразовать обратно веса и смещения.
    W1 = pos[0:5180].reshape((n_inputs,n_hidden))
    b1 = pos[5180:5550].reshape((n_hidden,))
    W2 = pos[5550:6660].reshape((n_hidden,n_classes))
    b2 = pos[6660:6663].reshape((n_classes,))

    # Выполнение прямого распространения
    z1 = X.dot(W1) + b1  # Преактивация в слое 1
    a1 = np.tanh(z1)     # Aктивация в слое 1
    z2 = a1.dot(W2) + b2 # Преактивация в слое 2
    logits = z2          # Логиты для слоя 2.

    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [12]:
(predict(X, pos) == y).mean()

0.43966282165039927

#### Попытка применения нейросети по [образцу](https://pyswarms.readthedocs.io/en/development/examples/custom_objective_function.html)

In [13]:
# Загрузка набора данных iris.
data = load_iris()

# Сохраним признаки в переменную X и метки в переменную y.
X = data.data
y = data.target

In [14]:
X.shape, y.shape

((150, 4), (150,))

In [15]:
X[0].shape, X[160:163]

((4,), array([], shape=(0, 4), dtype=float64))

In [16]:
# Прямое распространение
def forward_prop(params):
    """Прямое распространение как целевая функция

    то вычисляет прямое распространение нейронной сети, а также потерю. 
    Он получает набор параметров, которые должны быть преобразованы обратно 
    в соответствующие веса и смещения.

    Входные данные 
    ------
    params: np.ndarray
        Размерности должны включать развернутую версию весов и смещений.
    Возвращает 
    -------
    float
        Вычисленная потеря отрицательного логарифма правдоподобия на основе параметров"
    """
    # Архитектура нейронной сети
    n_inputs = 4
    n_hidden = 20
    n_classes = 3

    # Преобразовать обратно веса и смещения
    W1 = params[0:80].reshape((n_inputs,n_hidden))
    b1 = params[80:100].reshape((n_hidden,))
    W2 = params[100:160].reshape((n_hidden,n_classes))
    b2 = params[160:163].reshape((n_classes,))

    # Выполнение прямого распространения
    z1 = X.dot(W1) + b1  # Преактивация в слое 1
    a1 = np.tanh(z1)     # Активация в слое 1
    z2 = a1.dot(W2) + b2 # Преактивация в слое 2
    logits = z2          # Активация в слое 2

    # Вычисление софтмакс для логитов
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Вычисление отрицательной логарифмической функциии правдоподобия"
    N = 150 # Количество выборок
    corect_logprobs = -np.log(probs[range(N), y])
    loss = np.sum(corect_logprobs) / N

    return loss

In [17]:
def f(x):
    """Более высокоуровневый метод для выполнения прямого распространения во всем рое.
    Входные данные
    ------
    x: numpy.ndarray формы (n_particles, dimensions)
        Рой, которая будет выполнять поиск.

    Возвращает
    -------
    numpy.ndarray формы (n_particles, )
        Рассчитанная ошибка для каждой частицы."
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [18]:
# Инициализировать роевой алгоритм.
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Вызов экземпляра метода PSO
dimensions = (4 * 20) + (20 * 3) + 20 + 3
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Производим оптимизацию
cost, pos = optimizer.optimize(objective_func=f, iters=1000, verbose=3)

2023-07-17 20:08:50,143 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0.0167
2023-07-17 20:09:16,926 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.016707792584817856, best pos: [ 2.12250207e-02 -2.44845659e+00  5.45244510e-02  3.43632934e-01
  1.46979696e+00 -1.33790078e+00 -2.55690565e-01 -5.38755875e-02
 -2.90750202e-01 -2.40284581e-01 -2.47165769e-01  1.56102898e+00
  1.07994805e+00  6.86582629e-01  4.34812111e-01  5.50903660e-01
 -3.29546330e-02  5.61361610e-01 -2.15849666e+00  2.74973457e+00
 -4.00439231e-02  7.90912862e-01  1.06898901e+00  5.59940720e-03
  4.40758309e-01 -4.40404646e-03 -2.16243600e+00  2.53089281e+00
 -4.94778459e-01 -3.02724413e+00 -1.04152155e-01 -9.97809207e-01
 -7.14625698e-01 -8.01319513e-01 -3.23297116e-02 -1.27151303e+00
  8.63220269e-01 -6.63231686e-01 -6.97831685e-01 -3.39071868e-01
 -5.82367670e-01 -3.88

In [19]:
def predict(X, pos):
    """
    Используем обученные веса для выполнения предсказаний классов.

    Входные данные
    ------
    X: numpy.ndarray
        Входной набор данных Iris
    pos: numpy.ndarray
        Матрица позиций, найденных роем. Будет преобразована в веса
        и смещения.
    """
    # Архитектура нейронной сети
    n_inputs = 4
    n_hidden = 20
    n_classes = 3

    #Преобразовать обратно веса и смещения
    W1 = pos[0:80].reshape((n_inputs,n_hidden))
    b1 = pos[80:100].reshape((n_hidden,))
    W2 = pos[100:160].reshape((n_hidden,n_classes))
    b2 = pos[160:163].reshape((n_classes,))

    # Выполнение прямого распространения
    z1 = X.dot(W1) + b1  #  Преактивация в слое 1
    a1 = np.tanh(z1)     #  Активация в слое 1
    z2 = a1.dot(W2) + b2 #  Преактивация в слое 2
    logits = z2          #  Активация в слое 2

    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [20]:
(predict(X, pos) == y).mean()

0.9866666666666667

### Сделайте краткий обзор научной работы, посвящённой алгоритму нейронных сетей, 	 не рассматриваемому ранее на курсе. Проведите анализ: чем отличается выбранная архитектура от других? В чём плюсы и минусы данной архитектуры? Какие могут возникнуть трудности при её применении на практике?

Particle swarm optimizer эффективный механизм, исходя из статей, [статья ](https://learn.microsoft.com/ru-ru/archive/msdn-magazine/2011/august/artificial-intelligence-particle-swarm-optimization) ,[статья ](https://www.researchgate.net/publication/363840097_A_Concise_Overview_of_Particle_Swarm_Optimization_Methods) Для дополнительного понимания были использованы некотые материалы<br>
- Shi Y., Eberhart R.C. Parameter selection in particle swarm optimization, in: International conference
on evolutionary programming,Springer, 1998 DOI:10.1007/BFb0040810, pp. 591–600.
- Marandi A., Afshinmanesh F., Shahabadi M., Bahrami F. Boolean particle swarm optimization and
its application to the design of a dual-band dual-polarized planar antenna, 2006 IEEE Interna-
tional Conference on Evolutionary Computation, IEEE, 2006 DOI: 10.1109/CEC.2006.1688716,
pp. 3212-3218
- Hamada M., Hassan M. Artiﬁcial neural networks and particle swarm optimization algorithms for
preference prediction in multicriteria recommender systems,Informatics, 5, Multidisciplinary Dig-
ital Publishing Institute, 2018 DOI: 10.3390/informatics5020025. vol. 5, no. 2, pp. 25.
- Манусов В. З., Матренин П. В., Насрулло Х. Применение алгоритмов роевого интеллекта
в управлении генерирующим потребителем с возобновляемыми источниками энергии,Сист.
анал. и обработ. данных,2019 DOI: 10.17212/1814-1196-2019-3-115-134. Т. 76, № 3, С. 115-13
- Частикова В. А., Власов К. А., Картамышев Д. А. Обнаружение DDoS-атак на основе нейронных
сетей с применением метода роя частиц в качестве алгоритма обучения, Фундаментальные
исследования, 2014. Т. 4, № 8, С. 829-832.
### Заключение 
В заключение можно сказать, что PSO продемонстрировал преимущество во многих областях применения по сравнению со многими другими алгоритмами оптимизации. Некоторые недостатки, выявленные в алгоритме, были уменьшены различными модификациями канонического ω-PSO. Данный метод оптимизации является универсальным, что еще больше подчеркивает его эффективность. Такая гибкость сделала PSO надежным оптимизатором в самых разных, но сложных сценариях оптимизации. Методы оптимизации обладают значительной способностью
управлять задачами в сложных и обширных системах
Но для того что бы разобраться в этой не простой теме нужна постоянная практика и изучение дополнителной литературы(банально не хватает знаний). Очень сложная и емкая тема. 